## modal analysis

In [1]:
from numpy import *
from matplotlib.pyplot import *
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt
from scipy.linalg import expm
from numpy.linalg import *
from scipy.signal import lfilter
%matplotlib inline

In [2]:
def fileO(x,y,filename):
    f = open(filename,'w');
    #f = open(filename,'a'); # overwrite
    for i in range(len(y)):
        f.write('%.4e\t%.4e\n'%(x[i],y[i]));
    f.close();

In [3]:
def fileI(filename):
    tmp = []
    f = open(filename,'r');
    lines = f.readlines()
    for line in lines:
        tmp.append(line);
    out=np.zeros([len(tmp),2]);
    for i in range(len(out)):
        out[i,:] = np.array(tmp[i].split()).astype(np.float64)
    return out;

In [4]:
# SDOF Transient Response (ISO 18431-4)
# th_in : acceleration time history at base
# th_out : acceleration time history at mass
def SDOF_ISO(th_in,dt,fn,zeta):

    wn = 2*np.pi*fn;
    wd = wn*np.sqrt(1-zeta**2);
    A=wn*dt*zeta;
    B=wd*dt;
    
    a, b = np.zeros(3), np.zeros(3);

    b[0] = 1 - np.exp(-A)*np.sin(B)/B;
    b[1] = 2*np.exp(-A)*(np.sin(B)/B - np.cos(B));
    b[2] = np.exp(-2*A) - np.exp(-A) * np.sin(B)/B;

    a[0] = 1;
    a[1] = -2*np.exp(-A)*np.cos(B);
    a[2] = np.exp(-2*A);

    th_out=lfilter(b, a, th_in, axis=-1, zi=None);
    
    return th_out;

In [5]:
## SRS (ISO 18431-4)
def srs_iso(ft,th,dr,dt):
    nf = len(ft);
    rs = np.zeros(nf);
    for i in range(nf):
        rs[i]= max(abs(SDOF_ISO(th,dt,ft[i],dr)));
    return ft,rs;

In [6]:
ndof = 10;

In [7]:
M = 100*identity(ndof+1)

In [8]:
M[ndof,ndof] = 1e12;

In [9]:
K = np.zeros([ndof+1,ndof+1]);

In [10]:
kk = np.array([[1,-1],[-1,1]]);

In [11]:
for i in range(ndof):
    K[i:i+2,i:i+2] += kk;

In [12]:
K *= 1e6

In [13]:
eigval,eigvec = eig(inv(M)@K);
idx = eigval.argsort();
eigval = eigval[idx];
eigvec = eigvec[:,idx];
wn = np.sqrt(eigval);
fn = wn/(2*np.pi);
phi = eigvec
Mn = phi.T@M@phi
Phi = np.zeros([ndof+1,ndof+1]);
for i in range(ndof+1):
    Phi[:,i] = phi[:,i]/np.sqrt(Mn[i,i]);
delta = np.ones([ndof+1,1]);
Gamma = Phi.T@M@delta

In [14]:
# print modal analysis result
for i in range(ndof+1):
    print('Mode : %d \t fn = %.4f Hz \t P.F = %.4f'%(i+1,fn[i],Gamma[i]));

Mode : 1 	 fn = 0.0000 Hz 	 P.F = 1000000.0005
Mode : 2 	 fn = 2.3787 Hz 	 P.F = -0.0000
Mode : 3 	 fn = 7.0831 Hz 	 P.F = 0.0000
Mode : 4 	 fn = 11.6292 Hz 	 P.F = 0.0000
Mode : 5 	 fn = 15.9155 Hz 	 P.F = 0.0000
Mode : 6 	 fn = 19.8463 Hz 	 P.F = -0.0000
Mode : 7 	 fn = 23.3338 Hz 	 P.F = -0.0000
Mode : 8 	 fn = 26.3000 Hz 	 P.F = -0.0000
Mode : 9 	 fn = 28.6787 Hz 	 P.F = -0.0000
Mode : 10 	 fn = 30.4168 Hz 	 P.F = 0.0000
Mode : 11 	 fn = 31.4755 Hz 	 P.F = 0.0000


In [15]:
# mass normalized mode shape
# Phi

## check input time history (EL CENTRO EARTHQUAKE)

In [16]:
data = fileI('elcentro.dat');
time,th = data[:,0], data[:,1];
dt = time[1]-time[0]; N = len(th);

In [17]:
ft = np.logspace(-1,2,100); damp=1/20;

In [18]:
freq,rs_iso = srs_iso(ft,gth[:,1],damp,dt);

NameError: name 'gth' is not defined

In [ ]:
plt.plot(freq,rs_iso,'k-');
plt.xscale('log'); plt.yscale('log');
plt.xlim([0.1,20]); plt.ylim([0.01,1]);
plt.title('SHOCK RESPONSE SPECTRA, Q=10\n EL CENTRO EARTHQUAKE MAY 18, 1940 \n NORTH-SOUTH COMPONENT');
plt.xlabel('NATURAL FREQUENCY (Hz)');
plt.ylabel('PEAK ACCELERATION (G)');
plt.gca().xaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f')); 
plt.gca().yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'));
plt.grid();

In [ ]:
plt.plot(freq,rs_iso,'k-');
plt.xscale('log'); plt.yscale('log');
plt.xlim([0.1,100]); plt.ylim([0.01,1]);
plt.title('SHOCK RESPONSE SPECTRA, Q=10\n EL CENTRO EARTHQUAKE MAY 18, 1940 \n NORTH-SOUTH COMPONENT');
plt.xlabel('NATURAL FREQUENCY (Hz)');
plt.ylabel('PEAK ACCELERATION (G)');
plt.gca().xaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f')); 
plt.gca().yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1f'));
plt.grid();

In [ ]:
fzpa = 20;

## time history analysis (mode superposition, Newmark)

In [ ]:
def newmark_parameter(dt):
    r = 0.005; alpha = (1/4)*((1+r)**2); delta = 0.5 + r;
    a = np.zeros(8);
    a[0] = 1/(alpha*dt*dt); a[1] = delta/(alpha*dt); a[2] = 1/(alpha*dt); a[3] = 1/(2*alpha)-1;
    a[4] = delta/alpha - 1; a[5] = (dt/2)*((delta/alpha)-2); a[6] = dt*(1-delta); a[7] = delta*dt;
    return a;

In [ ]:
a = newmark_parameter(dt);

In [ ]:
# th_in : acceleration time history at base
# th_out : acceleration time history at mass
def SDOF_newmark(th_in,dt,fn,zeta):
    def newmark_parameter(dt):
        r = 0.005; alpha = (1/4)*((1+r)**2); delta = 0.5 + r;
        a = np.zeros(8);
        a[0] = 1/(alpha*dt*dt); a[1] = delta/(alpha*dt); a[2] = 1/(alpha*dt); a[3] = 1/(2*alpha)-1;
        a[4] = delta/alpha - 1; a[5] = (dt/2)*((delta/alpha)-2); a[6] = dt*(1-delta); a[7] = delta*dt;
        return a;
    def DERIV(X,Y):
        N = len(Y); YY = np.zeros(N);
        YY[0] = (Y[1]-Y[0])/(X[1]-X[0]); Y[N-1] = (Y[N-1]-Y[N-2])/(X[N-1]-X[N-2]);
        for n in range(N-2):
            YY[n+1] = (((Y[n+2]-Y[n+1])/(X[n+2]-X[n+1]))*(X[n+1]-X[n]) + ((Y[n+1]-Y[n])/(X[n+1]-X[n]))*(X[n+2]-X[n+1]))/(X[n+2]-X[n]);
        return YY;
    def INTEG(X,Y,C1):
        N = len(Y); YY = np.zeros(N);
        YY[0] = C1;
        for n in range(1,N):
            YY[n] = YY[n-1] + (1/2)*(Y[n]+Y[n-1])*(X[n]-X[n-1]);
        return YY;      
    a = newmark_parameter(dt);
    u,v,ac = np.zeros(len(th_in)),np.zeros(len(th_in)),np.zeros(len(th_in));
    ru,rv,rac = np.zeros(len(th_in)),np.zeros(len(th_in)),np.zeros(len(th_in));
    N = len(th_in);
    time = np.arange(N)*dt;
    vth = INTEG(time,th_in,0); dth = INTEG(time,vth,0);
    F = -th_in;
    wn = 2*np.pi*fn; m = 1; c = 2*zeta*wn; k = wn**2;            
    Fe = a[0]*m + a[1]*c + k;          
    for n in range(len(th_in)-1):
        ru[n+1] = (F[n+1] + m*(a[0]*ru[n]+a[2]*rv[n]+a[3]*rac[n])+c*(a[1]*ru[n]+a[4]*rv[n]+a[5]*rac[n]))/Fe;
        rv[n+1] = a[1]*(ru[n+1]-ru[n]) - a[4]*rv[n] - a[5]*rac[n];
        rac[n+1] = a[0]*(ru[n+1]-ru[n]) -a[2]*rv[n] - a[3]*rac[n];
    th_out= rac + th_in;
    return th_out

In [ ]:
zeta = 0.02*np.ones(ndof+1);

In [ ]:
# modal coordinate MCK matrices
Mn = Phi.T@M@Phi;
Cn = 2*np.diag(zeta)@np.diag(wn);
Kn = Phi.T@K@Phi;

In [ ]:
# set initial disp, vel, acc matrices
disp, vel, acc = np.zeros([ndof+1,N]),np.zeros([ndof+1,N]),np.zeros([ndof+1,N]);

In [ ]:
# effective stiffness matrix
Ke = a[0]*Mn+a[1]*Cn+Kn;

In [ ]:
# input force matrix
forc = M[ndof,ndof]*th;
F = np.zeros([ndof+1,N]);
F[ndof,:] = forc;
F = inv(phi.T@M@phi)@phi.T@F;

In [ ]:
# run Newmark integration loop
for i in range(ndof):
    Fe = F[:,i+1]+Mn@(a[0]*disp[:,i]+a[2]*vel[:,i]+a[3]*acc[:,i])+Cn@(a[1]*disp[:,i]+a[4]*vel[:,i]+a[5]*acc[:,i])
    disp[:,i+1] = inv(Ke)@Fe
    vel[:,i+1] = a[1]*(disp[:,i+1]-disp[:,i]) - a[4]*vel[:,i] - a[5]*acc[:,i]
    acc[:,i+1] = a[0]*(disp[:,i+1]-disp[:,i]) - a[2]*vel[:,i] - a[3]*acc[:,i]

In [ ]:
x = phi@disp